# Implementation of False Positive Algorithms

In [48]:
import json
import mysql.connector
import sql_commands
import random
import numpy as np
#random.seed(2311)

In [49]:
db_names = ['tpch_2pct', 'tpch_4pct', 'tpch_6pct', 'tpch_8pct', 'tpch_10pct']
null_rates = [0.02, 0.04, 0.06, 0.08, 0.1]

In [50]:
mydb, mycursor = sql_commands.connect_to_db("tpch_og")

In [51]:
# all_nations = sql_commands.run_query(mycursor, "SELECT N_NAME FROM NATION")
# def get_rand_nation():
#     index = random.randint(0, len(all_nations)-1)
#     nation = all_nations[index][0]
#     #print("Nation chosen:", nation)
#     return nation
nations = ['UNITED KINGDOM', 'ARGENTINA', 'PERU', 'FRANCE', 'BRAZIL']

In [54]:
def q1_fps(query1_out,lineitem_cols,query1_cols):
    # Query to get all items from Lineitem table
    lineitem = sql_commands.run_query(mycursor,"SELECT * FROM LINEITEM")
    # Variables needed for comparing lineitem items to query 1 answer
    l_orderkey_index = lineitem_cols.index('L_ORDERKEY')
    #print("l-orderkey-index", l_orderkey_index)
    o_orderkey_index = query1_cols.index('O_ORDERKEY')
    #print("o-orderkey-index", o_orderkey_index)
    l_suppkey_index = lineitem_cols.index('L_SUPPKEY')
    s_suppkey_index = query1_cols.index("S_SUPPKEY")
    l_commitdate_index = lineitem_cols.index("L_COMMITDATE")
    l_receiptdate_index = lineitem_cols.index('L_RECEIPTDATE')
    count = 0
    continue_count = 0
    not_count = 0
    nothing = 0
    matching_count = 0
    for i,t in enumerate(lineitem):
        for j,a in enumerate(query1_out):
            #print("Lineitem orderkey:", t[l_orderkey_index], "query1 answer orderkey:", a[o_orderkey_index])
            if t[l_orderkey_index] == a[o_orderkey_index]:
                matching_count += 1
                x = t[l_suppkey_index]
                if x is not None and x == a[s_suppkey_index]:
                    continue_count += 1
                    continue
                
                d1 = t[l_commitdate_index]
                d2 = t[l_receiptdate_index]
                if (d1 is None) or (d2 is None) or (d2 > d1):
                    count += 1
                else:
                    not_count += 1
            else:
                nothing += 1
    #print(not_count, matching_count) 
    if matching_count != 0:           
        return count/matching_count
    return None

In [55]:
# list of tuples
# (suppkey, orderkey)
q1_results = []
print("QUERY 1 False Positives")
for i in range(len(db_names)):
    curr_db_name = db_names[i]
    curr_null_rate = null_rates[i]
    print(f"\nNull Rate: {curr_null_rate}")
    mydb, mycursor = sql_commands.connect_to_db(curr_db_name)
    fp_pcts = []
    for j in range(3):
        nation = nations[j]
        print(f"Nation: {nation}")
        query1_out = sql_commands.run_query1(mycursor,nation)
        lineitem_attributes = sql_commands.get_attribute_names(mycursor,"LINEITEM")
        query1_attributes = ['S_SUPPKEY', 'O_ORDERKEY']
        fp_pct = q1_fps(query1_out, lineitem_attributes, query1_attributes)
        #print(f"{fp_pct*100:.4}% false positives")
        #print(f"Total Number of rows in query: {len(query1_out)}")
        if fp_pct != None:
            fp_pcts.append(fp_pct)

    #print(fp_pcts)
    print(f"Average false positive rate is: {np.mean(fp_pcts)}")
    q1_results.append(np.mean(fp_pcts))
    

QUERY 1 False Positives

Null Rate: 0.02
Nation: UNITED KINGDOM
Nation: ARGENTINA
Nation: PERU
Average false positive rate is: 0.09128052742823267

Null Rate: 0.04
Nation: UNITED KINGDOM
Nation: ARGENTINA
Nation: PERU
Average false positive rate is: 0.1506480897382979

Null Rate: 0.06
Nation: UNITED KINGDOM
Nation: ARGENTINA
Nation: PERU
Average false positive rate is: 0.21371327948268773

Null Rate: 0.08
Nation: UNITED KINGDOM
Nation: ARGENTINA
Nation: PERU
Average false positive rate is: 0.26871189086092767

Null Rate: 0.1
Nation: UNITED KINGDOM
Nation: ARGENTINA
Nation: PERU
Average false positive rate is: 0.3253795296924006


In [57]:
q1_results

[0.09128052742823267,
 0.1506480897382979,
 0.21371327948268773,
 0.26871189086092767,
 0.3253795296924006]

In [ ]:
sql_commands.close_db(mydb, mycursor)